In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
op_headers = ["anzahl", "code", "ebm", "kuerzel", "betriebsstaette", "text", "seite", "datum", "patientid", "vorname", "nachname", "geburtsdatum", "jahre", "scheinid", "geschlecht"]

In [3]:
m17_data = pd.read_csv("../data/M17_Data.csv")
m17_diag = pd.read_csv("../data/M17_Diag.csv")
m17_op = pd.read_csv("../data/M17_Operations.csv")
m17_op.columns = op_headers

m22_data = pd.read_csv("../data/M22_Data.csv")
m22_diag = pd.read_csv("../data/M22_Diag.csv")
m22_op = pd.read_csv("../data/M22_Operations.csv")
m22_op.columns = op_headers

m23_data = pd.read_csv("../data/M23_Data.csv")
m23_op = pd.read_csv("../data/M23_Operations.csv")
m23_diag = pd.read_csv("../data/M23_Diag.csv")
m23_op.columns = op_headers

s83_data = pd.read_csv("../data/S83_Data.csv")
s83_op = pd.read_csv("../data/S83_Operations.csv")
s83_diag = pd.read_csv("../data/S83_Diag.csv")
s83_op.columns = op_headers

Dataframe preparatation:
1. Concatenate all sub-dataframes of the same type
2. Reset index
3. Drop the unnecessary columns to anonymize and reduce dimensions
4. (Optional) Add any additionaly columns to make patient records, operations and diagnoses similar in format
5. Sort by date
6. (Optional) Filter a column
7. Remove any duplicates and optionally ignore a column while doing it

In [4]:
def prep_df(to_concat, to_drop, to_add="", to_filter=[], to_ignore_in_duplicates=[]):
    df = pd.concat(to_concat, ignore_index=True)
    df = df.reset_index(drop=True)
    df = df.drop(to_drop, axis ="columns", inplace=False)
    if len(to_add) > 0: df[to_add] = ""
    if "datum" in df.columns:
        df["datum"] = pd.to_datetime(df["datum"], dayfirst=True, errors="coerce")
        df = df.sort_values(by="datum")
    if len(to_filter) > 1:
        df = df[df[to_filter[0]].isin(to_filter[1])] 
    df = df.drop_duplicates(subset=df.columns.difference(to_ignore_in_duplicates))
    df = df.reset_index(drop=True)
    return df

# Patient Records

In [5]:
anam_exam = prep_df(to_concat=[m17_data, m22_data, m23_data, s83_data],
                       to_drop=["typ", "betriebsstaette", "nachname", "vorname", "letzter_nutzer", "karteieintragid", "medientyp"],
                       to_add="code",
                       to_filter=["karteityp", ["ANA", "BEF"]],
                       to_ignore_in_duplicates=["karteityp"])

In [6]:
anam_exam.head()

,datum,text,karteityp,patientid,geschlecht,code
0,1900-01-01,Z.n. Ac li Knie mit retropatellarer Knorperlgl...,ANA,63707,female,
1,1900-01-01,Schmerzen bei Treppensteigen; nach längerem Si...,ANA,28581,female,
2,1900-01-01,Seit ca. 3 Monaten Schmerzen re. Knie; Ruhesch...,ANA,25929,male,
3,1900-01-01,29.05 :FB: Anfrage der Versorgungsamtes zum ...,BEF,4972,male,
4,1900-01-01,Seit 2 Jahren immer mal wieder Schmerzen in li...,ANA,62119,female,


# Operations

In [7]:
ops = prep_df(to_concat=[m17_op, m22_op, m23_op, s83_op],
                       to_drop=["anzahl", "ebm", "kuerzel", "seite", "vorname", "nachname", "scheinid", "betriebsstaette"],
                       to_add="karteityp")
ops["karteityp"] = "OP"

In [8]:
ops.head()

,code,text,datum,patientid,geburtsdatum,jahre,geschlecht,karteityp
0,5-812.6,Arthroskopische Operation am Gelenkknorpel und...,2005-04-04,55162,11.09.1979,44,female,OP
1,5-812.7,Arthroskopische Operation am Gelenkknorpel und...,2005-04-04,55153,01.10.1967,56,male,OP
2,5-812.6,Arthroskopische Operation am Gelenkknorpel und...,2005-04-04,53783,01.04.1957,67,female,OP
3,5-812.6,Arthroskopische Operation am Gelenkknorpel und...,2005-04-04,55160,14.06.1946,78,female,OP
4,5-812.6,Arthroskopische Operation am Gelenkknorpel und...,2005-04-04,55157,10.08.1988,35,female,OP


# Diagnoses

In [9]:
diag = prep_df(to_concat=[m17_diag, m22_diag, m23_diag, s83_diag],
                       to_drop=["anzahl", "kuerzel", "ddi", "vorname", "nachname", "scheinid"],
                       to_add="karteityp")
diag["karteityp"] = "DIA"
diag.rename(columns={"eigene_bezeichnung":"text"}, inplace=True)


In [10]:
diag.head()

,code,icdtext,text,datum,patientid,geburtsdatum,jahre,geschlecht,karteityp
0,M22.4 G,Chondromalacia patellae,rechts,1996-01-04,113,21.09.1981,42,female,DIA
1,M23.0- G,Meniskusganglion,Binnenschädigung des Kniegelenkes (internal de...,1996-01-04,150,29.10.1966,57,male,DIA
2,M23.0- G,Meniskusganglion,Binnenschädigung des Kniegelenkes (internal de...,1996-01-04,93,04.05.1977,47,male,DIA
3,M17.9 G,Gonarthrose; nicht näher bezeichnet,Gonarthrose re.; med. Meniskusläsion re.; lat....,1996-01-04,105,08.06.1930,94,female,DIA
4,S83.2 G,Akuter Meniskusriss,Meniskusriß; akut,1996-01-04,93,04.05.1977,47,male,DIA


In [11]:
icd_text = diag[['code', 'icdtext']].copy()
icd_text['code'] = icd_text['code'].str.strip()
icd_dict = icd_text.drop_duplicates().to_dict('records')
icd_dict = {item['code']: item['icdtext'] for item in icd_dict}

In [12]:
icd_dict #icd_code: icdtext

{'M22.4 G': 'Chondromalacia patellae',
 'M23.0- G': 'Meniskusganglion',
 'M17.9 G': 'Gonarthrose; nicht näher bezeichnet',
 'S83.2 G': 'Akuter Meniskusriss',
 'M23.3-': 'Sonstige Meniskusschädigungen',
 'M17.9 A': 'Gonarthrose; nicht näher bezeichnet',
 'M23.39 G': 'Sonstige Meniskusschädigungen: nicht näher bezeichneter Meniskus',
 'S83.53 V': 'Verstauchung und Zerrung des Kniegelenkes mit Riss des vorderen Kreuzbandes',
 'M23.23 G': 'Meniskusschädigung durch alten Riss oder alte Verletzung: sonstiger und nicht näher bezeichneter Teil des Innenmeniskus',
 'M22.2 G': 'Krankheiten im Patellofemoralbereich',
 'M23.24 G': 'Meniskusschädigung durch alten Riss oder alte Verletzung: Vorderhorn des Außenmeniskus',
 'M23.59 G': 'Chronische Instabilität des Kniegelenkes: nicht näher bezeichnetes Band ',
 'M17.1 G': 'Sonstige primäre Gonarthrose',
 'M23.33 G': 'Sonstige Meniskusschädigungen: sonstiger und nicht näher bezeichneter Teil des Innenmeniskus',
 'S83.5- Z': 'Verstauchung und Zerrung de

In [13]:
birthdays = diag[['patientid', 'geburtsdatum']].drop_duplicates().to_dict('records')
birthdays = {item['patientid']: item['geburtsdatum'] for item in birthdays}

In [14]:
birthdays # patientid: birthday

{113: '21.09.1981',
 150: '29.10.1966',
 93: '04.05.1977',
 105: '08.06.1930',
 104: '03.04.1920',
 110: '22.05.1954',
 118: '10.04.1937',
 79: '10.02.1977',
 70: '09.08.1935',
 69: '06.08.1967',
 177: '16.04.1963',
 278: '08.08.1967',
 282: '16.08.1931',
 350: '07.01.1945',
 492: '01.11.1936',
 521: '27.06.1969',
 501: '06.09.1966',
 568: '06.04.1939',
 609: '03.06.1956',
 602: '24.04.1940',
 731: '09.06.1971',
 799: '18.11.1970',
 768: '23.04.1950',
 939: '17.01.1945',
 903: '06.03.1950',
 695: '28.02.1963',
 924: '14.06.1951',
 919: '29.07.1939',
 920: '16.03.1964',
 926: '23.09.1933',
 716: '18.10.1943',
 1061: '28.12.1973',
 821: '04.03.1968',
 1163: '02.04.1961',
 909: '17.12.1957',
 900: '21.04.1925',
 842: '05.03.1969',
 1261: '28.12.1966',
 1230: '25.05.1957',
 1278: '09.10.1954',
 1275: '24.10.1948',
 1308: '25.03.1927',
 1297: '02.03.1959',
 1313: '21.05.1970',
 1130: '01.05.1976',
 1077: '25.10.1933',
 1315: '13.06.1939',
 429: '15.04.1950',
 1394: '31.03.1975',
 194: '11.0

# Merge all together

### Diagnosis final pre-processing
If the icdtext is longer than the eigenbezeichnung (text), substitute text value with the icdtext value

In [15]:
diag.head()

,code,icdtext,text,datum,patientid,geburtsdatum,jahre,geschlecht,karteityp
0,M22.4 G,Chondromalacia patellae,rechts,1996-01-04,113,21.09.1981,42,female,DIA
1,M23.0- G,Meniskusganglion,Binnenschädigung des Kniegelenkes (internal de...,1996-01-04,150,29.10.1966,57,male,DIA
2,M23.0- G,Meniskusganglion,Binnenschädigung des Kniegelenkes (internal de...,1996-01-04,93,04.05.1977,47,male,DIA
3,M17.9 G,Gonarthrose; nicht näher bezeichnet,Gonarthrose re.; med. Meniskusläsion re.; lat....,1996-01-04,105,08.06.1930,94,female,DIA
4,S83.2 G,Akuter Meniskusriss,Meniskusriß; akut,1996-01-04,93,04.05.1977,47,male,DIA


In [16]:
diag_final = diag.copy()

diag_final['icdtext'] = diag_final['icdtext'].astype(str)
diag_final['text'] = diag_final['text'].astype(str)

diag_final['text'] = diag_final.apply(lambda row: row['icdtext'] if len(row['icdtext']) > len(row['text']) else row['text'], axis=1)
diag_final = diag_final.drop(columns=["icdtext"])

In [17]:
diag_final.head()

,code,text,datum,patientid,geburtsdatum,jahre,geschlecht,karteityp
0,M22.4 G,Chondromalacia patellae,1996-01-04,113,21.09.1981,42,female,DIA
1,M23.0- G,Binnenschädigung des Kniegelenkes (internal de...,1996-01-04,150,29.10.1966,57,male,DIA
2,M23.0- G,Binnenschädigung des Kniegelenkes (internal de...,1996-01-04,93,04.05.1977,47,male,DIA
3,M17.9 G,Gonarthrose re.; med. Meniskusläsion re.; lat....,1996-01-04,105,08.06.1930,94,female,DIA
4,S83.2 G,Akuter Meniskusriss,1996-01-04,93,04.05.1977,47,male,DIA


### Patient Records final pre-processing
Fill in missing years and birthdays

In [18]:
anam_exam_final = anam_exam.copy()

anam_exam_final['geburtsdatum'] = ""
anam_exam_final['geburtsdatum'] = anam_exam_final['patientid'].map(birthdays)
anam_exam_final['geburtsdatum'] = pd.to_datetime(anam_exam_final['geburtsdatum'], dayfirst=True)

# Calculate 'jahre'
anam_exam_final['jahre'] = (abs(anam_exam_final['datum'] - anam_exam_final['geburtsdatum'])).dt.days // 365

In [19]:
anam_exam_final.head()

,datum,text,karteityp,patientid,geschlecht,code,geburtsdatum,jahre
0,1900-01-01,Z.n. Ac li Knie mit retropatellarer Knorperlgl...,ANA,63707,female,,1962-03-14,62.0
1,1900-01-01,Schmerzen bei Treppensteigen; nach längerem Si...,ANA,28581,female,,1950-07-18,50.0
2,1900-01-01,Seit ca. 3 Monaten Schmerzen re. Knie; Ruhesch...,ANA,25929,male,,1943-03-28,43.0
3,1900-01-01,29.05 :FB: Anfrage der Versorgungsamtes zum ...,BEF,4972,male,,1973-09-02,73.0
4,1900-01-01,Seit 2 Jahren immer mal wieder Schmerzen in li...,ANA,62119,female,,1954-07-01,54.0


### Operations final pre-processing

In [20]:
ops_final = ops.copy()

### Merge All

In [21]:
all = pd.concat([diag_final, ops_final, anam_exam_final], ignore_index=True)
all.reset_index()
all = all.drop(["geburtsdatum"], axis="columns")

In [45]:
grouped_lists = all.sort_values(["datum"]).groupby('patientid').agg(list)

def lists_to_dicts(row, pid):
    return [{'patientid': pid, **{col: row[col][i] for col in grouped_lists.columns}} for i in range(len(row[grouped_lists.columns[0]]))]

grouped_lists['entries'] = grouped_lists.apply(lambda row: lists_to_dicts(row, row.name), axis=1)

grouped_df = pd.DataFrame({
    'patientid': grouped_lists.index,
    'entries': grouped_lists['entries']
}).reset_index(drop=True)

In [46]:
grouped_df

,patientid,entries
0,61,"[{'patientid': 61, 'code': '', 'text': 'Klinis..."
1,67,"[{'patientid': 67, 'code': '', 'text': 'Re. Ac..."
2,68,"[{'patientid': 68, 'code': 'M23.23 V', 'text':..."
3,69,"[{'patientid': 69, 'code': 'M23.39 G', 'text':..."
4,70,"[{'patientid': 70, 'code': 'M23.3- ', 'text': ..."
...,...,...
42993,701652,"[{'patientid': 701652, 'code': '', 'text': 'Z...."
42994,701654,"[{'patientid': 701654, 'code': '', 'text': 'Vo..."
42995,701695,"[{'patientid': 701695, 'code': 'M23.59 G', 'te..."
42996,701711,"[{'patientid': 701711, 'code': '', 'text': 'Am..."


In [47]:
from datetime import datetime, timedelta

In [48]:
# Function to group entries by the 2-year criterion
def group_entries(entries):
    if not entries:
        return []
    
    grouped_entries = []
    current_group = [entries[0]]

    for entry in entries[1:]:
        if entry['datum'] >= current_group[-1]['datum'] + timedelta(days=365*2):
            grouped_entries.append(current_group)
            current_group = [entry]
        else:
            current_group.append(entry)
    
    grouped_entries.append(current_group)
    return grouped_entries

# Apply the grouping function
grouped_df['grouped_entries'] = grouped_df['entries'].apply(group_entries)
grouped_df = grouped_df.drop(columns=['entries'])

In [49]:
grouped_df.iloc[0]['grouped_entries']

[[{'patientid': 61,
   'code': '',
   'text': 'Klinischer Befund des Unterschenkels re.: deutliche Schwellung. an der medialen Tibiakante; leichte Verhärtung der medialen Wadenmuskulatur.',
   'datum': Timestamp('1996-01-03 00:00:00'),
   'jahre': 30.0,
   'geschlecht': 'male',
   'karteityp': 'BEF'},
  {'patientid': 61,
   'code': '',
   'text': 'Klinischer Befund des Unterschenkels re.: insgesamt besser;',
   'datum': Timestamp('1996-01-05 00:00:00'),
   'jahre': 30.0,
   'geschlecht': 'male',
   'karteityp': 'BEF'},
  {'patientid': 61,
   'code': '',
   'text': 'Zwischenanamnese: Beschwerden unverändert.',
   'datum': Timestamp('1996-01-05 00:00:00'),
   'jahre': 30.0,
   'geschlecht': 'male',
   'karteityp': 'ANA'},
  {'patientid': 61,
   'code': '',
   'text': 'Bef. re. Leiste: Subcutane Narbe wulstig;',
   'datum': Timestamp('1996-06-27 00:00:00'),
   'jahre': 30.0,
   'geschlecht': 'male',
   'karteityp': 'BEF'},
  {'patientid': 61,
   'code': '',
   'text': 'Immer noch Probleme

In [50]:
sum(len(groups) for groups in grouped_df['grouped_entries'])

51243

In [64]:
structured_data = pd.DataFrame(columns=['patientid', 'sex', 'age', 'ANA', 'EXA', 'DIA_code', 'DIA_text', 'OP_code', 'OP_text'])
def has_X(type, dict):
    return dict['karteityp'] == type

def get_elements(entry, arg):
    if entry is not None: return entry[arg]
    else: return None
        

def structure_occurrence(group):
    ana_elem = next((entry for entry in group if has_X('ANA', entry)), None)
    bef_elem = next((entry for entry in group if has_X('BEF', entry)), None)
    op_elem = next((entry for entry in group if has_X('OP', entry)), None)
    dia_elem = next((entry for entry in group if has_X('DIA', entry)), None)
    
    patientid = get_elements(ana_elem, 'patientid')
    sex = get_elements(ana_elem, 'geschlecht')
    age = get_elements(ana_elem, 'jahre')
    ANA = get_elements(ana_elem, 'text')
    EXA = get_elements(bef_elem, 'text')
    DIA = get_elements(dia_elem, 'text')
    DIA_code = get_elements(dia_elem, 'code')
    OP = get_elements(op_elem, 'text')
    OP_code = get_elements(op_elem, 'code')
    
    return {'patientid': patientid, 'sex': sex, 'age': age, 'ANA': ANA, 'EXA': EXA, 'DIA_text': DIA, 'DIA_code': DIA_code, 'OP_text': OP, 'OP_code': OP_code}
    
flattened_entries = []
for groups in grouped_df['grouped_entries']:
    for group in groups:
        flattened_entries.append(structure_occurrence(group))

# Create a new DataFrame from the flattened entries
structured_data = pd.DataFrame(flattened_entries)

In [65]:
structured_data

,patientid,sex,age,ANA,EXA,DIA_text,DIA_code,OP_text,OP_code
0,61.0,male,30.0,Zwischenanamnese: Beschwerden unverändert.,Klinischer Befund des Unterschenkels re.: deut...,None,None,None,None
1,61.0,male,32.0,Beschwerden re. Achillessehne. NMR unauffällig...,Re. Achillessehne: Verdickung im Bereich der A...,Retropatellare Chondromalazie li; Insertionste...,M22.4 G,None,None
2,67.0,male,41.0,Seit mehreren Monaten Beschwerden re. Achilles...,Re. Achillessehne: Deutliche Auftreibung der A...,V.a.laterale Chondromalazie bei st.n. Außenmen...,M23.3- V,None,None
3,NaN,None,NaN,None,None,Meniskusschädigung durch alten Riss oder alte ...,M23.23 V,None,None
4,NaN,None,NaN,None,None,Sonstige Meniskusschädigungen: nicht näher bez...,M23.39 G,None,None
...,...,...,...,...,...,...,...,...,...
51238,701652.0,female,51.0,Z.n. mehrfachen KnieOP´s (4x LCA + IM-TR); let...,Gangbild flüssig; Beinachse varisch 1-2 QF; Be...,"mediale Gonarthrose ""bone to bone"" links",M17.1 G,None,None
51239,701654.0,male,16.0,Vor 6 Wochen Kniedistorsion beim FB. Valgus-Ve...,Gangbild flüssig; Beinachse varisch 1-2 QF; Be...,Verstauchung und Zerrung des Kniegelenkes mit ...,S83.51 G,None,None
51240,701695.0,female,42.0,Z.n. VKB-Plastik vor 3 Jahren; keine Instabili...,Gangbild flüssig; Beinachse valgisch 1-2 QF; B...,Chronische Instabilität des Kniegelenkes: nich...,M23.59 G,None,None
51241,701711.0,male,16.0,Am 24.9. beim Fußballspielen nach einem Kopfba...,Gangbild flüssig; Beinachse varisch 1-2 QF; Be...,Sonstige Meniskusschädigungen: Hinterhorn des ...,M23.35 G,None,None


In [69]:
fs_data = structured_data.copy()

In [70]:
fs_data = fs_data.dropna(subset=['ANA', 'EXA'], how='all')   
fs_data = fs_data[fs_data['DIA_code'].notna()]

In [71]:
len(fs_data[fs_data['patientid'].notna()])

44449

In [77]:
midpoint = len(fs_data) // 2

# Split the DataFrame
test = fs_data.copy().iloc[:midpoint]
test2 = fs_data.copy().iloc[midpoint:]

In [78]:
print(len(fs_data))
print(len(test))
print(len(test2))

45257
22628
22629


# Examination Interpretation

In [75]:
import IO
import pandas as pd
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')

/Users/juliankraus/Library/CloudStorage/OneDrive-Personal/Uni/Languages/Code/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/juliankraus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juliankraus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Textual Adaptions

In [ ]:
# List of strings that should not be split
do_not_split = [
    "li.",
    "bds.",
    "med.",
    "neg.",
    "lat.",
    'pos.',
    "re.",
    "mm, links",
    "mm, rechts",
    "mm, li",
    "mm, re",
    "o.B",
    "mm, verletzt",
    "mm, Seitendifferenz",
    "diskr.",
    "flüssig und mit"
]
def custom_split(text, exceptions):
    for exception in exceptions:
        text = re.sub(re.escape(exception), re.sub(r'[^\w\s]', '', exception), text, flags=re.IGNORECASE)
    parts = re.split(r'[.,]|\sund\s', text)
    parts = [part.strip() for part in parts if part.strip()]
    return parts

In [ ]:
# Preprocess the text
pattern_pre = r'\d{1,3}/\d{1,3}/\d{1,3}|\d{1,3}-\d{1,3}-\d{1,3}'

def adaptText(match):
    # Extract the matched text
    matched_text = match.group(0)
    
    # Split the matched text by either '/' or '-'
    if '/' in matched_text:
        parts = matched_text.split('/')
    elif '-' in matched_text:
        parts = matched_text.split('-')
    
    # Convert parts to integers
    parts = list(map(int, parts))
    
    # Evaluate each part in the context of the "Neutral-Null-Methode"
    hyperextension = parts[0]
    extension_deficit = parts[1]
    flexion = parts[2]
    
    # Create descriptions based on the measurements
    if hyperextension > 0:
        hyperextension_desc = "überstreckung des knies ist möglich,"
    else:
        hyperextension_desc = "keine überstreckung des knies,"
    
    if extension_deficit > 0:
        extension_deficit_desc = "streckdefizit des knies,"
    else:
        extension_deficit_desc = "kein streckdefizit des knies,"
    
    if flexion >= 120:
        flexion_desc = "gute beugung des knies,"
    elif flexion >= 90:
        flexion_desc = "angemessene beugung des knies,"
    else:
        flexion_desc = "eingeschränkte beugung des knies,"
    
    # Combine the descriptions into a final string
    result = f",{hyperextension_desc} {extension_deficit_desc} {flexion_desc}"
    return result

def preprocess(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub('ds', 'druckschmerz', text, flags=re.IGNORECASE)
    text = re.sub('medial', 'innere', text, flags=re.IGNORECASE)
    text = re.sub('gelenkspalt', 'seite des kniegelenks', text, flags=re.IGNORECASE)



    text = re.sub('lateral', 'äußere', text, flags=re.IGNORECASE)
    text = re.sub('lat', 'äußere', text, flags=re.IGNORECASE)
    text = re.sub('gs', 'seite des kniegelenks', text, flags=re.IGNORECASE)
    text = re.sub('li ', 'links ', text, flags=re.IGNORECASE)
    text = re.sub('re ', 'rechts ', text, flags=re.IGNORECASE)
    text = re.sub('gb ', 'gangbild ', text, flags=re.IGNORECASE)
    text = re.sub('patellafacette', 'kniescheibe', text, flags=re.IGNORECASE)
    text = re.sub('patella', 'kniescheibe', text, flags=re.IGNORECASE)


    text = re.sub(pattern_pre, adaptText, text, flags=re.IGNORECASE)

    text = re.sub(r'[^\w\s/-]', '', text)
    return text

In [ ]:
patterns = [
    r'^\d{1,3}/\d{1,3}/\d{1,3}$',  # Pattern for 1-3 digits/numbers/1-3 digits/numbers/1-3 digits/numbers
    r'^\d{1,3}-\d{1,3}-\d{1,3}$',  # Pattern for 1-3 digits-numbers-1-3 digits-numbers-1-3 digits-numbers
    r'.*gangbild.*',               
    r'.*druck.*',                 
    r'.*ergu.*',              
    r'.*schwell.*',       
    r'.*rötung.*',          
    r'.*überstreckung.*',     
]

# Combine patterns into a single regular expression
combined_pattern = r'|'.join(patterns)

In [ ]:
inter_df = test
# Filter the list
for idx, row in inter_df.iterrows():
    text = row['EXA']
    if text is not None:
        parts = custom_split(row['EXA'], do_not_split)  # Custom split by exceptions
        parts = list(map(preprocess, parts))
        
        filtered_parts = [s for s in parts if s and re.match(combined_pattern, s, re.IGNORECASE)]
        
        inter_df.at[idx, 'EXA_formatted'] = ', '.join(filtered_parts)
    else:
        inter_df.at[idx, 'EXA_formatted'] = ""

## Formulation

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import IO

In [ ]:
# Load the Mistral model and tokenizer from Hugging Face
model_name = "jphme/em_german_leo_mistral"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create a text generation pipeline
text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0, return_full_text=False)

In [ ]:
few_shot_examples = [
    ("links knie diskr erguss, kein anteromedialer druckschmerz med äußerechts Seite des Kniegelenks, kein druckschmerz kniescheiben, keine meniskuszeichen rechts knie kein erguss, druckschmerz seitlich kniescheibe", 
     "Das linke Knie hat keinen Erguss, und es gibt keine Schmerzen beim Drücken auf der äußerechts Seite des Kniegelenks und Kniescheibe. Das rechte Knie hat auch keinen Erguss, aber an der Seite der Kniescheibe, gibt es Schmerzen beim Drücken."),
    ("rechts knie deutlicher erguss, keine überstreckung des knies, streckdefizit des knies, angemessene beugung des knies, keine rötung", 
     "Das rechte Knie hat einen deutlichen Erguss, es ist nicht möglich das Knie zu überstrecken und auch nicht es komplett zu strecken. Die Beugung des Knies ist nur angemessen möglich. Das Knie ist nicht gerötet."),
]
# Construct the few-shot prompt
instruction = "Du bist ein Patient, formuliere die Beschreibung deiner Symptome aus."
few_shot_prompt = f"{instruction}\n\n" + "\n\n".join([f"USER: {inp}\nASSISTANT: {out}" for inp, out in few_shot_examples])
for idx, row in inter_df.iterrows():
    text = row['EXA_formatted']
    if text != "":
        # Combine the text and instruction
        prompt = f"{few_shot_prompt}\n\nUSER: {row['Formatted']} \n ASSISTANT:"
        # Generate the output
        generated_text = text_generation_pipeline(prompt, max_new_tokens=200)[0]['generated_text']
        output_text = generated_text
        inter_df.at[idx, 'EXA_interpreted'] = output_text
    else:
        inter_df.at[idx, 'EXA_interpreted'] = ""
inter_df.to_csv("../data/interpreted_data.csv")